In [1]:
# import necessari
import librosa
import numpy
import sys
import os
import glob
import itertools
import re
import pickle
import keras
import tensorflow as tf
import tensorflow.keras.models
import tensorflow.keras.backend as K
import keras.optimizers
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Activation, Flatten, Multiply, Add, Reshape
from tqdm import tqdm
from itertools import groupby
from keras.utils import to_categorical
from sklearn import metrics

# costanti 
ALPHA = 0.75
N_MELS = 128
HOP_LENGTH = 512
N_FFT = 1024
POWER = 2.0
FRAME_NUMS = 313
NUM_FILES = 3349
FRAMES = 10

# FEATURES EXTRACTION

# Loading da Google Drive
train_data = numpy.load("/content/drive/MyDrive/DCASE_DATA_EXTRACTED/train/training_pump.npy")

In [2]:
# Estrazione spettrogrammi divisi per ID
id_00 = train_data[0:906]
id_02 = train_data[906:1811]
id_04 = train_data[1811:2413]
id_06 = train_data[2413:3349]

# Z-Score Normalization ID_00
id_00_norm = numpy.empty_like(id_00)
max_00 = numpy.max(id_00)
min_00 = numpy.min(id_00)
id_00_norm = (id_00 - min_00) / (max_00 - min_00)

# Z-Score Normalization ID_02
id_02_norm = numpy.empty_like(id_02)
max_02 = numpy.max(id_02)
min_02 = numpy.min(id_02)
id_02_norm = (id_02 - min_02) / (max_02 - min_02)

# Z-Score Normalization ID_04
id_04_norm = numpy.empty_like(id_04)
max_04 = numpy.max(id_04)
min_04 = numpy.min(id_04)
id_04_norm = (id_04 - max_04) / (max_02 - min_02)

# Z-Score Normalization ID_06
id_06_norm = numpy.empty_like(id_06)
max_06 = numpy.max(id_06)
min_06 = numpy.min(id_06)
id_06_norm = (id_06 - max_06) / (max_06 - min_06)
'''
print("Mean: {m}".format(m=mean_00))
print("Dev.Std: {d}".format(d=std_00))
print(id_00_norm.shape)

print("Mean: {m}".format(m=mean_02))
print("Dev.Std: {d}".format(d=std_02))
print(id_02_norm.shape)

print("Mean: {m}".format(m=mean_04))
print("Dev.Std: {d}".format(d=std_04))
print(id_04_norm.shape)

print("Mean: {m}".format(m=mean_06))
print("Dev.Std: {d}".format(d=std_06))
print(id_06_norm.shape)
'''
train_data_norm = numpy.concatenate([id_00_norm, id_02_norm, id_04_norm, id_06_norm])

In [3]:
# Estrazione frame 128x10 da ciascun spettrogramma
training_splitted = numpy.zeros((len(train_data_norm)*30, 128, 10)) # Dataset utilizzato per il training
index = 0
for vector_array in train_data_norm:
  i = 0
  while (i+5) < 303:
    vector_i = numpy.zeros((128,10))
    for j in range(0,128):
      vector_i[j] = vector_array[j][i:i+10]
    training_splitted[index] = vector_i
    index += 1
    i = i+10

In [4]:
training_splitted.shape

(200940, 128, 10)

In [4]:
training = numpy.zeros((len(training_splitted), 10, 128))
index = 0
for elem in training_splitted:
  training[index] = elem.T
  index += 1
training.shape

(100470, 10, 128)

In [5]:
# Shuffling
randomize = numpy.arange(len(training))
numpy.random.shuffle(randomize)
training_tot_shuffle = training[randomize]
print(training_tot_shuffle.shape)


(100470, 10, 128)


In [6]:
timesteps = 10
num_features = 128

model = keras.Sequential()
model.add(keras.layers.LSTM(64, kernel_initializer='he_uniform', batch_input_shape=(None, timesteps, num_features), return_sequences=True, name='encoder_1'))
model.add(keras.layers.LSTM(32, kernel_initializer='he_uniform', return_sequences=True, name='encoder_3'))
model.add(keras.layers.LSTM(16, kernel_initializer='he_uniform', return_sequences=False, name='encoder_4'))
model.add(keras.layers.RepeatVector(timesteps, name='encoder_decoder_bridge'))
model.add(keras.layers.LSTM(16, kernel_initializer='he_uniform', return_sequences=True, name='decoder_1'))
model.add(keras.layers.LSTM(32, kernel_initializer='he_uniform', return_sequences=True, name='decoder_2'))
model.add(keras.layers.LSTM(64, kernel_initializer='he_uniform', return_sequences=True, name='decoder_3'))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(num_features)))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_1 (LSTM)             (None, 10, 64)            49408     
_________________________________________________________________
encoder_3 (LSTM)             (None, 10, 32)            12416     
_________________________________________________________________
encoder_4 (LSTM)             (None, 16)                3136      
_________________________________________________________________
encoder_decoder_bridge (Repe (None, 10, 16)            0         
_________________________________________________________________
decoder_1 (LSTM)             (None, 10, 16)            2112      
_________________________________________________________________
decoder_2 (LSTM)             (None, 10, 32)            6272      
_________________________________________________________________
decoder_3 (LSTM)             (None, 10, 64)            2

In [7]:
opt = keras.optimizers.Adam(
    learning_rate = 0.0001,
    beta_1=0.95,
    beta_2=0.999
)

model.compile(loss="mse",optimizer = opt)
model.fit(training_tot_shuffle, training_tot_shuffle, epochs=50, validation_split=0.1, batch_size=64, shuffle=True)

Epoch 1/50
1413/1413 [==============================] - 76s 27ms/step - loss: 0.0945 - val_loss: 0.0083
Epoch 2/50
1413/1413 [==============================] - 36s 25ms/step - loss: 0.0067 - val_loss: 0.0049
Epoch 3/50
1413/1413 [==============================] - 36s 25ms/step - loss: 0.0048 - val_loss: 0.0045
Epoch 4/50
1413/1413 [==============================] - 35s 25ms/step - loss: 0.0045 - val_loss: 0.0040
Epoch 5/50
1413/1413 [==============================] - 35s 25ms/step - loss: 0.0039 - val_loss: 0.0036
Epoch 6/50
1413/1413 [==============================] - 36s 25ms/step - loss: 0.0035 - val_loss: 0.0034
Epoch 7/50
1413/1413 [==============================] - 36s 25ms/step - loss: 0.0034 - val_loss: 0.0034
Epoch 8/50
1413/1413 [==============================] - 36s 25ms/step - loss: 0.0034 - val_loss: 0.0033
Epoch 9/50
1413/1413 [==============================] - 36s 25ms/step - loss: 0.0033 - val_loss: 0.0032
Epoch 10/50
1413/1413 [==============================] - 36s 25m

In [8]:
model.save('/content/drive/MyDrive/models/LSTM-4/model_pump.h5')

In [9]:
import csv

def save_csv(save_file_path,
             save_data):
    with open(save_file_path, "w", newline="") as f:
        writer = csv.writer(f, lineterminator='\n')
        writer.writerows(save_data)


# load dataset
def select_dirs(path):
    dir_path = os.path.abspath(path)
    dirs = sorted(glob.glob(dir_path))
    return dirs

def file_load(wav_name, mono=False):
    try:
        return librosa.load(wav_name, sr=None, mono=mono)
    except:
        logger.error("file_broken or not exists!! : {}".format(wav_name))

def file_list_generator(target_dir, dir_name="train", ext="wav"):
    print("target_dir : {}".format(target_dir))

    # generate training list
    training_list_path = os.path.abspath("{dir}/{dir_name}/*.{ext}".format(dir=target_dir, dir_name=dir_name, ext=ext))
    files = sorted(glob.glob(training_list_path))
    if len(files) == 0:
      print("errore")
    return files


def file_to_vector_array(file_name, n_mels=64, n_fft=1024, hop_length=512, power=2.0):
    # 02 generate melspectrogram using librosa
    y, sr = file_load(file_name)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, power=power)

    # 03 convert melspectrogram to log mel energy
    log_mel_spectrogram = 20.0 / power * numpy.log10(mel_spectrogram + sys.float_info.epsilon)

    return log_mel_spectrogram

  
def list_to_vector_array(file_list, msg="calc...", n_mels=64, n_fft=1024, hop_length=512, power=2.0, frames=10):
    # iterate file_to_vector_array()
    for idx in tqdm(range(len(file_list)), desc=msg):
        vector_array = file_to_vector_array(file_list[idx], n_mels=n_mels, n_fft=n_fft, hop_length=hop_length, power=power)

        if idx == 0:
            dataset = numpy.zeros((len(file_list), n_mels, frames), float)
        dataset[vector_array.shape[0] * idx: vector_array.shape[0] * (idx + 1), :] = vector_array
    return dataset

def key_by_id(item):
  path_splitted = item.split("/")
  file_name = path_splitted[ len(path_splitted) - 1 ]
  file_name_splitted = file_name.split("_")
  machine_id = file_name_splitted = file_name_splitted[2]
  return machine_id

def get_machine_id_list_for_test(target_dir,
                                 dir_name="test",
                                 ext="wav"):

    # create test files
    dir_path = os.path.abspath("{dir}/{dir_name}/*.{ext}".format(dir=target_dir, dir_name=dir_name, ext=ext))
    file_paths = sorted(glob.glob(dir_path))
    # extract id
    machine_id_list = sorted(list(set(itertools.chain.from_iterable(
        [re.findall('id_[0-9][0-9]', ext_id) for ext_id in file_paths]))))
    return machine_id_list

def test_file_list_generator(target_dir,
                             id_name,
                             dir_name="test",
                             prefix_normal="normal",
                             prefix_anomaly="anomaly",
                             ext="wav"):
  
    print("target_dir : {}".format(target_dir+"_"+id_name))

    normal_files = sorted(
    glob.glob("{dir}/{dir_name}/{prefix_normal}_{id_name}*.{ext}".format(dir=target_dir,
                                                                                 dir_name=dir_name,
                                                                                 prefix_normal=prefix_normal,
                                                                                 id_name=id_name,
                                                                                 ext=ext)))
    normal_labels = numpy.zeros(len(normal_files))
    anomaly_files = sorted(
    glob.glob("{dir}/{dir_name}/{prefix_anomaly}_{id_name}*.{ext}".format(dir=target_dir,
                                                                                  dir_name=dir_name,
                                                                                  prefix_anomaly=prefix_anomaly,
                                                                                  id_name=id_name,
                                                                                  ext=ext)))
    anomaly_labels = numpy.ones(len(anomaly_files))
    files = numpy.concatenate((normal_files, anomaly_files), axis=0)
    labels = numpy.concatenate((normal_labels, anomaly_labels), axis=0)
    print("test_file  num : {num}".format(num=len(files)))
    if len(files) == 0:
        print("no_wav_file!!")
    print("\n========================================")

    return files, labels

In [ ]:
target_dir = "/content/drive/MyDrive/pump"

machine_type = os.path.split(target_dir)[1]
print("============== MODEL LOAD ==============")
# set model path
model_file = "/content/drive/MyDrive/models/LSTM-4/model_pump.h5"

# load model file
if not os.path.exists(model_file):
  print("{} model not found ".format(machine_type))
  sys.exit(-1)
model = keras.models.load_model(model_file)
# model.summary()

machine_id_list = get_machine_id_list_for_test(target_dir)

# initialize lines in csv for AUC and pAUC
csv_lines = []

csv_lines.append([machine_type])
csv_lines.append(["id", "AUC", "pAUC"])
performance = []

for id_str in machine_id_list:
  # load test file

  id_num = id_str.split("_")[1]

  # Definizione della label "match" da utilizzare in fase di testing e del min e max da utilizzare per la normalizzazione
  # i min e max sono stati calcolati a partire dai dati di training.
  if id_num == "00":
    max = max_00
    min = min_00
  if id_num == "02":
    max = max_02
    min = min_02
  if id_num == "04":
    max = max_04
    min = min_04
  if id_num == "06":
    max = max_06
    min = min_06

  test_files, y_true = test_file_list_generator(target_dir, id_str)

  # setup anomaly score file path
  anomaly_score_csv = "/content/drive/MyDrive/models/LSTM-4/anomaly_score_{machine_type}_{id_str}.csv".format(machine_type=machine_type, id_str=id_str)
  anomaly_score_list = []

  print("\n============== BEGIN TEST FOR A MACHINE ID {id} ==============".format(id=id_num))

  y_pred = [0. for k in test_files]


  for file_idx, file_path in tqdm(enumerate(test_files), total=len(test_files)):

    # Estrazione spettrogramma audio test
    data = file_to_vector_array(file_path, n_mels=N_MELS, n_fft=N_FFT, hop_length=HOP_LENGTH, power=POWER)

    # Normalizzazione spettrogramma di test
    data = ( data - min ) / (max-min)

    # Estrazione delle frame 128x10
    data_splitted = numpy.zeros((30, 128, 10))
    index = 0
    i = 0
    while (i+5) < 303:
      vector_i = numpy.zeros((128,10))
      for j in range(0,128):
        vector_i[j] = data[j][i:i+10]
      data_splitted[index] = vector_i
      index += 1
      i = i+10

    elem_error = []
    for elem in data_splitted:
      predicted = model.predict(elem.reshape(1,10,128))
      errors = numpy.mean( numpy.square( elem.T - predicted.reshape((1,10,128)) ) , axis=1)
      elem_error.append(numpy.mean(errors))

    # Log dell'errore associato all'istanza di test
    y_pred[file_idx] = numpy.mean(elem_error)
    anomaly_score_list.append([os.path.basename(file_path), y_pred[file_idx]])
  
  save_csv(save_file_path=anomaly_score_csv, save_data=anomaly_score_list)
    
  # Calcolo AUC e pAUC per i dati con un certo ID_0x
  auc = metrics.roc_auc_score(y_true,y_pred)
  p_auc = metrics.roc_auc_score(y_true, y_pred, max_fpr=0.1)
  csv_lines.append([id_str.split("_", 1)[1], auc, p_auc])
  performance.append([auc, p_auc])
  print("AUC : {}".format(auc))
  print("pAUC : {}".format(p_auc))

  print("\n============ END OF TEST FOR A MACHINE ID ============")

# Stampa di AUC e pAUC medi su tutti i dati di test (media di AUC e pAUC sui vari ID).
print("\n============ AVERAGE PERFORMANCES ============")
averaged_performance = numpy.mean(numpy.array(performance, dtype=float), axis=0)
csv_lines.append(["Average"] + list(averaged_performance))
csv_lines.append([])
print(averaged_performance)

result_path = "/content/drive/MyDrive/models/LSTM-4/anomaly_score_avg.csv"
save_csv(save_file_path=result_path, save_data=csv_lines)

============== MODEL LOAD ==============


  0%|          | 0/243 [00:00<?, ?it/s]

target_dir : /content/drive/MyDrive/pump_id_00
test_file  num : 243


============== BEGIN TEST FOR A MACHINE ID 00 ==============


 59%|█████▉    | 144/243 [05:17<03:37,  2.20s/it]